![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 3.4: Functional Programming
#### Written by Stevo Bailey ([stevo@berkeley.edu](mailto:stevo@berkeley.edu))

## Table of Contents
Scala is a functional programming language. This means functions can be passed around as objects, avoiding mutable data in favor of computation. You can use functional programming techniques when constructing your hardware generators to improve the capabilities and flexibility. This module presents an introduction to functional programming use in Chisel.

**[Functional Programming in Scala](#scala)**
1. [Functions as Objects](#obj)
1. [Anonymous Functions](#anon)
1. [Functions Exercises](#scalaex)

**[Functional Programming in Chisel](#chisel)**
1. [Packages and Imports](#packages)


# Functional Programming in Scala<a name="scala"></a>
Scala functions were introduced in Module 1. Functions take any number of inputs and produce one output. Inputs are often called arguments to a function. To produce no output, return the `Unit` type. Below are some examples of functions.

In [ ]:
// no inputs or outputs
def hello(): Unit = { print("Hello!") }

// math operation, one input and one output
def times2(x: Int): Int = { 2 * x }

// inputs can have default values, and the return type is optional
def timesN(x: Int, n: Int = 2) = { n * x }

// call the functions listed above
hello()
times2(4)
timesN(4)         // no need to specify n to use the default value
timesN(4, 3)      // argument order is the same as the order where the function was defined
timesN(n=7, x=2)  // arguments may be reordered and assigned to explicitly

## Functions as Objects <a name="obj"></a>
Functions may be treated like objects. That means we can assign a function to a `val` and pass it to classes, objects, or other functions as an argument.

In [ ]:
// these are normal functions
def plus1funct(x: Int): Int = { x + 1 }
def times2funct(x: Int): Int = { x * 2 }

// these are functions as vals
// the first one explicitly specifies the return type
val plus1val: Int => Int = x => { x + 1 }
val times2val = (x: Int) => { x * 2 }

// calling both looks the same
plus1funct(4)
plus1val(4)

Why would you want to create a `val` instead of a `def`? With a `val`, you can now pass the function around to other functions, as shown below. You can even create your own functions that accept other functions as arguments.

In [ ]:
// create our function
val plus1 = (x: Int) => x + 1
val times2 = (x: Int) => x * 2

// pass it to map, a list function
val myList = List(1, 2, 5, 9)
val myListPlus = myList.map(plus1)
val myListTimes = myList.map(times2)


// create a custom function, which performs an operation on X N times using recursion
def opN(x: Int, n: Int, op: Int => Int): Int = {
  if (n <= 0) { x }
  else { opN(op(x), n-1, op) }
}

opN(7, 3, plus1)
opN(7, 3, times2)

## Anonymous Functions <a name="anon"></a>
As the name implies, anonymous functions are nameless. There's no need to create a `val` for a function if we'll only use it once. The following example demonstrates this

In [ ]:
val myList = List(1, 2, 5, 9)

// add one to every item in the list using anonymous functions
// these all do the same thing
myList.map( (x:Int) => x + 1 )
myList.map(_ + 1)

## Functions Exercises<a name="scalaex"></a>
TODO

# Functional Programming in Chisel<a name="chisel"></a>
Let's look at some examples of how to use functional programming when creating hardware generators in Chisel.

In [23]:
// Run this boilerplate for the necessary imports

import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::firrtl:1.0-SNAPSHOT_2017-07-19`
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import $ivy.$                                                 

import $ivy.$                                                          

import $ivy.$                                                

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

First, we'll revisit the FIR filter from previous examples. Instead of passing in the coefficients as parameters to the class or making them programmable, we'll pass a function to the FIR that defines how the window coefficients are calculated. This function will take the window length and bitwidth to produce a scaled list of coefficients. Here are two example windows. To avoid fractions, we'll scale the coefficients to be between the max and min integer values. For more on these windows, check out the [this Wikipedia page](https://en.wikipedia.org/wiki/Window_function).

In [25]:
// get some math functions
import scala.math.{abs, round, cos, Pi, pow}

// simple triangular window
def TriangularWindow(length: Int, bitwidth: Int): Seq[Int] = {
  val raw_coeffs = (0 until length).map( (x:Int) => 1-abs((x.toDouble-(length-1)/2.0)/((length-1)/2.0)) )
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

// Hamming window
def HammingWindow(length: Int, bitwidth: Int): Seq[Int] = {
  val raw_coeffs = (0 until length).map( (x: Int) => 0.54 - 0.46*cos(2*Pi*x/(length-1)))
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

// check it out!
TriangularWindow(length=10, bitwidth=16)
HammingWindow(length=10, bitwidth=16)

import scala.math.{abs, round, cos, Pi}

// simple triangular window

defined function TriangularWindow
defined function HammingWindow
res24_3: Seq[Int] = Vector(0, 4, 7, 11, 14, 14, 11, 7, 4, 0)
res24_4: Seq[Int] = Vector(1, 3, 7, 12, 16, 16, 12, 7, 3, 1)

Now we'll create a FIR filter that accepts a window function as the argument. This allows us to define new windows later on and retain the same FIR generator. It also allows us to independently size the FIR, knowing the window will be recalculated for different lengths or bitwidths.

In [32]:
// our FIR has parameterized window length, IO bitwidth, and windowing function
class MyFir(length: Int, bitwidth: Int, window: (Int, Int) => Seq[Int]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitwidth.W))
    val out = Output(UInt(bitwidth.W))
  })

  // calculate the coefficients using the provided window function, mapping to UInts
  val coeffs = window(length, bitwidth).map(_.U)
  
  // create an array holding the output of the delays
  val delays = Vec(length, UInt(bitwidth.W)).scanLeft(io.in)( (prev: UInt, next: UInt) => {
    next := RegNext(prev)
    next
  })
  
  // multiply, putting result in "mults"
  val mults = delays.zip(coeffs).map{ case(delay: UInt, coeff: UInt) => delay * coeff }
  
  // add up multiplier outputs
  val result = mults.reduceLeft(_+_)
  
  // choose MSBs
  io.out := result(result.getWidth-1, result.getWidth-bitwidth)
}

defined class MyFir

In [ ]:
import scala.math.{pow, sin, Pi}


// test parameters
val length = 10
val bitwidth = 16
val window = TriangularWindow

// test our FIR
Driver(() => new MyFir(length, bitwidth, window)) {
  c => new PeekPokeTester(c) {
    
    // test data
    val n = 1000
    val freq = 10
    
    // sample data
    val max_value = pow(2, bitwidth)-1
    val sine = (0 until n).map(i => max_value/2 + max_value/2*sin(2*Pi*freq*i))
    
    // 
  }
}